# 06 Backtesting
- Simulate strategy vs. benchmark
- Cumulative returns, Sharpe Ratio
- Performance summary

# 06 Backtesting
This notebook simulates the performance of the optimized portfolio and compares it to a benchmark portfolio.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load processed price data for TSLA, BND, SPY
assets = ['TSLA', 'BND', 'SPY']
dfs = []
for asset in assets:
    df = pd.read_csv(f'../data/processed/{asset}_processed.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date')
    dfs.append(df['Adj Close'])
prices = pd.concat(dfs, axis=1)
prices.columns = assets
prices = prices.dropna()

# Set backtest period (last year)
start_date = '2024-08-01'
end_date = '2025-07-31'
prices_bt = prices.loc[start_date:end_date]
prices_bt.head()

In [ ]:
# Define strategy and benchmark weights
# Example: use max Sharpe weights from optimization (replace with your actual weights)
strategy_weights = {'TSLA': 0.4, 'BND': 0.3, 'SPY': 0.3}
benchmark_weights = {'TSLA': 0.0, 'BND': 0.4, 'SPY': 0.6}

# Calculate daily returns
returns = prices_bt.pct_change().dropna()

# Calculate portfolio returns
strat_returns = sum(returns[asset] * w for asset, w in strategy_weights.items())
bench_returns = sum(returns[asset] * w for asset, w in benchmark_weights.items())

In [ ]:
# Calculate and plot cumulative returns for strategy and benchmark
strat_cum = (1 + strat_returns).cumprod()
bench_cum = (1 + bench_returns).cumprod()

plt.figure(figsize=(12,6))
plt.plot(strat_cum, label='Strategy Portfolio')
plt.plot(bench_cum, label='Benchmark (60% SPY, 40% BND)')
plt.title('Backtest: Cumulative Returns')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.show()

In [ ]:
# Calculate Sharpe ratio and total return for both portfolios
sharpe_strat = strat_returns.mean() / strat_returns.std() * np.sqrt(252)
sharpe_bench = bench_returns.mean() / bench_returns.std() * np.sqrt(252)
total_strat = strat_cum.iloc[-1] - 1
total_bench = bench_cum.iloc[-1] - 1

print(f"Strategy Sharpe: {sharpe_strat:.2f}, Total Return: {total_strat:.2%}")
print(f"Benchmark Sharpe: {sharpe_bench:.2f}, Total Return: {total_bench:.2%}")

## Summary & Interpretation
- Did your strategy outperform the benchmark?
- What does this initial backtest suggest about the viability of your model-driven approach?
- Discuss any limitations and next steps for further validation.